In [1]:
import os
import sqlite3
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from urllib.request import urlopen
from bs4 import BeautifulSoup as soup

import seaborn as sns
sns.set()

In [2]:
url = "http://www.producepriceindex.com/"

In [3]:
urls = []
for i in range(1, 638):
    urls.append(f'http://www.producepriceindex.com/produce-price-index?field_ppi_commodity_target_id=All&field_ppi_date_value%5Bmin%5D=&field_ppi_date_value%5Bmax%5D=&page={i}')

In [4]:

produce = dict()

for page in urls:
    uClient = urlopen(page) # Opens first page
    html = uClient.read() # Reads in all HTML for that page
    uClient.close() # Close connection
    page_soup = soup(html, 'html.parser') # Create soup object
    data = page_soup.find_all('td') # Grab the table data section
    

    for j in range(0, 25):
        commodity = []
        for i in data[8*j].contents[0].split(' '): # Pick out the name of the commodity (happens every 8 iterations)
            if (i != '') & (i != '\n'):
                commodity.append(i) # Pick out only the name, ditch the white spaces and newline


        # If the commodity name is one word long, set the dictionary with that word. If not already set, create an empty list
        if len(commodity) == 1: 
            word = commodity[0]
            if word not in produce:
                produce[word] = []
            produce[word].append(data[1 + 8*j].span.string) # append the date to the list. Found at position 1 and every 8 iterations thereafter

        # otherwise if the commodity is 2 words, fix this into a single item to serve as key. Then do the same as above
        elif len(commodity) == 2: 
            word = commodity[0] + ' ' + commodity[1]
            if word not in produce:
                produce[word] = []
            produce[word].append(data[1 + 8*j].span.string)

        # 3 words...
        else:
            word = commodity[0] + ' ' + commodity[1] + ' ' + commodity[2]
            if word not in produce:
                produce[word] = []
            produce[word].append(data[1 + 8*j].span.string)

        


        # Now collect the values for each commodity key
        # Everything repeats after 8 iterations, hence the 8*j.
        # data[2 + 8*j] holds the farm price
        for i in data[2 + 8*j].string.split(' '):
            if (i != '') & (i != '\n'):
                produce[word].append(i)

        # data[3 + 8*j] holds the Atlanta Retail Price
        for i in data[3 + 8*j].string.split(' '):
            if (i != '') & (i != '\n'):
                produce[word].append(i)

        # Holds the Chicago Retail Price
        for i in data[4 + 8*j].string.split(' '):
            if (i != '') & (i != '\n'):
                produce[word].append(i)

        # Los Angeles Retail Price
        for i in data[5 + 8*j].string.split(' '):
            if (i != '') & (i != '\n'):
                produce[word].append(i)


        # New York Retail Price
        for i in data[6 + 8*j].string.split(' '):
            if (i != '') & (i != '\n'):
                produce[word].append(i)


        # Average Spread
        for i in data[7 + 8*j].string.split(' '):
            if (i != '') & (i != '\n'):
                produce[word].append(i)


NOTE: Something weird is happening and I can't figure out why. The first time through, this scraped all the data correctly. Now it appears to be missing a few dates. It isn't a large amount, usually less than 3%. It will be ok for the analysis, but I would like to come back and fix it. For example, if you look at Potatoes for 2010-05-23 you will see there is an entry on the website. However, this is absent from the data.

In [5]:
dataframe_dict = {}
names = list(produce.keys())

for produce_name in names:
    dates = []
    farm_price = []
    atl = []
    chi = []
    la = []
    nyc = []
    inc = []
    number_of_rows = int(len(produce[produce_name])/7)
    

    for row_num in range(number_of_rows):

        dates.append(produce[produce_name][row_num*7])
        farm_price.append(produce[produce_name][(row_num*7+1)])
        atl.append(produce[produce_name][row_num*7+2])
        chi.append(produce[produce_name][row_num*7+3])
        la.append(produce[produce_name][row_num*7+4])
        nyc.append(produce[produce_name][row_num*7+5])
        inc.append(produce[produce_name][row_num*7+6])
    

    dictionary = {'Commodity': f'{produce_name}','Date': dates, 'FarmPrice': farm_price, 'AtlantaPrice': atl, 'ChicagoPrice': chi, 'LAPrice': la, 'NYCPrice': nyc, 'PercentIncrease':inc }
    dataframe_dict[produce_name] = pd.DataFrame(dictionary)

In [6]:
dedupe_dataframe_dict = {}
for produce_name in names:
    dedupe_dataframe_dict[produce_name] = dataframe_dict[produce_name].drop_duplicates()

In [7]:
for name in names:
    print(dedupe_dataframe_dict[name].isna().sum())

Commodity          0
Date               0
FarmPrice          0
AtlantaPrice       0
ChicagoPrice       0
LAPrice            0
NYCPrice           0
PercentIncrease    0
dtype: int64
Commodity          0
Date               0
FarmPrice          0
AtlantaPrice       0
ChicagoPrice       0
LAPrice            0
NYCPrice           0
PercentIncrease    0
dtype: int64
Commodity          0
Date               0
FarmPrice          0
AtlantaPrice       0
ChicagoPrice       0
LAPrice            0
NYCPrice           0
PercentIncrease    0
dtype: int64
Commodity          0
Date               0
FarmPrice          0
AtlantaPrice       0
ChicagoPrice       0
LAPrice            0
NYCPrice           0
PercentIncrease    0
dtype: int64
Commodity          0
Date               0
FarmPrice          0
AtlantaPrice       0
ChicagoPrice       0
LAPrice            0
NYCPrice           0
PercentIncrease    0
dtype: int64
Commodity          0
Date               0
FarmPrice          0
AtlantaPrice       0
ChicagoPric

In [8]:
dedupe_dataframe_dict['Strawberries']

,Commodity,Date,FarmPrice,AtlantaPrice,ChicagoPrice,LAPrice,NYCPrice,PercentIncrease
0,Strawberries,2019-05-05,$0.58,$2.42,$1.89,$2.12,$2.91,302.59%
1,Strawberries,2019-04-28,$0.58,$2.79,$1.95,$2.38,$2.96,334.48%
2,Strawberries,2019-04-21,$0.69,$2.92,$2.04,$2.38,$3.16,280.43%
3,Strawberries,2019-04-14,$0.90,$3.04,$2.31,$2.50,$3.41,212.78%
4,Strawberries,2019-04-07,$1.08,$3.04,$2.94,$2.62,$3.74,185.65%
...,...,...,...,...,...,...,...,...
979,Strawberries,1999-08-23,$0.72,$2.62,$0.00,$3.12,$2.59,189.24%
980,Strawberries,1999-08-16,$0.58,$2.44,$0.00,$3.22,$2.77,263.36%
981,Strawberries,1999-08-09,$0.65,$2.49,$0.00,$2.99,$2.87,221.15%
982,Strawberries,1999-08-02,$0.71,$2.37,$0.00,$2.87,$2.19,161.62%


In [9]:
df_dict = dict()

for name in names:
    dates = []
    farm_prices = []
    atl_prices = []
    chi_prices = []
    la_prices = []
    nyc_price = []


    dates = [pd.to_datetime(x[1]) for x in dedupe_dataframe_dict[name].values]
    farm_prices = [float(x[1:]) if (x[1:] != '') else np.nan for x in dedupe_dataframe_dict[name]['FarmPrice']]
    atl_prices = [float(x[1:]) if (x[1:] != '') else np.nan for x in dedupe_dataframe_dict[name]['AtlantaPrice']]
    chi_prices = [float(x[1:]) if (x[1:] != '') else np.nan for x in dedupe_dataframe_dict[name]['ChicagoPrice']]
    la_prices = [float(x[1:]) if (x[1:] != '') else np.nan for x in dedupe_dataframe_dict[name]['LAPrice']]
    nyc_prices = [float(x[1:]) if (x[1:] != '') else np.nan for x in dedupe_dataframe_dict[name]['NYCPrice']]


    temp_dict = {'Date': dates, 'FarmPrice': farm_prices, 'AtlantaPrice': atl_prices, 'ChicagoPrice': chi_prices, 'LAPrice':la_prices, 'NYCPrice': nyc_prices}
    df_dict[name] = pd.DataFrame(temp_dict)

In [10]:
cpi_df = pd.read_csv('../../data/00_raw/cpi.csv', index_col=0, header=1)

cpi_df.tail()

cpi_cols = ['1', '2', '3' , '4', '5', '6', '7', '8', '9', '10', '11', '12', 'Avg', 'Dec-Dec', 'Avg-Avg']

cpi_df.columns = cpi_cols

CPI_2020 = cpi_df.loc[2020][0] # Data frame is zero indexed... 0 is Jan and 1 is Feb etc for second argument.

CPI_2020

257.971

In [11]:
adj2020_df_dict = {}
for name in names:
    farm = []
    atl = []
    chi = []
    la = []
    nyc = []

    for row in df_dict[name].values:
        conversion_factor = CPI_2020/(cpi_df[str(row[0].month)][row[0].year])
        farm.append(round(row[1]*conversion_factor,2))
        atl.append(round(row[2]*conversion_factor,2))
        chi.append(round(row[3]*conversion_factor,2))
        la.append(round(row[4]*conversion_factor,2))
        nyc.append(round(row[5]*conversion_factor,2))

    adj2020_df = pd.DataFrame({'FarmPrice': farm, 'AtlantaPrice': atl, 'ChicagoPrice': chi, 'LAPrice': la, 'NYCPrice': nyc})
    adj2020_df.index = df_dict[name]['Date']
    adj2020_df_dict[name] = adj2020_df

In [12]:
for name in names:
    print('Percentage of Invalid Data for', name)
    print('FarmPrice: ', round(sum(adj2020_df_dict[name]['FarmPrice'] <= 0)/len(adj2020_df_dict[name])*100, 2), '%')
    print('ATL: ', round(sum(adj2020_df_dict[name]['AtlantaPrice'] <= 0)/len(adj2020_df_dict[name])*100, 2), '%')
    print('CHI: ', round(sum(adj2020_df_dict[name]['ChicagoPrice'] <= 0)/len(adj2020_df_dict[name])*100, 2), '%')
    print('LA: ', round(sum(adj2020_df_dict[name]['LAPrice'] <= 0)/len(adj2020_df_dict[name])*100, 2), '%')
    print('NYC: ', round(sum(adj2020_df_dict[name]['NYCPrice'] <= 0)/len(adj2020_df_dict[name])*100, 2), '%')
    print('\n')

Percentage of Invalid Data for Broccoli Bunches
FarmPrice:  0.0 %
ATL:  0.1 %
CHI:  8.28 %
LA:  0.2 %
NYC:  0.41 %


Percentage of Invalid Data for Avocados
FarmPrice:  0.0 %
ATL:  0.14 %
CHI:  8.87 %
LA:  0.28 %
NYC:  0.42 %


Percentage of Invalid Data for Strawberries
FarmPrice:  0.0 %
ATL:  0.0 %
CHI:  6.97 %
LA:  0.31 %
NYC:  0.2 %


Percentage of Invalid Data for Romaine Lettuce
FarmPrice:  0.0 %
ATL:  0.0 %
CHI:  7.95 %
LA:  0.1 %
NYC:  0.2 %


Percentage of Invalid Data for Red Leaf Lettuce
FarmPrice:  0.0 %
ATL:  0.0 %
CHI:  7.87 %
LA:  0.1 %
NYC:  0.3 %


Percentage of Invalid Data for Potatoes
FarmPrice:  0.35 %
ATL:  0.23 %
CHI:  9.47 %
LA:  0.0 %
NYC:  19.65 %


Percentage of Invalid Data for Oranges
FarmPrice:  0.12 %
ATL:  7.43 %
CHI:  8.98 %
LA:  1.44 %
NYC:  1.56 %


Percentage of Invalid Data for Iceberg Lettuce
FarmPrice:  0.0 %
ATL:  0.0 %
CHI:  7.95 %
LA:  0.1 %
NYC:  0.2 %


Percentage of Invalid Data for Green Leaf Lettuce
FarmPrice:  0.0 %
ATL:  0.0 %
CHI:  7.93

Some of the data frame columns hold up to 20% bad data. I am going to keep two versions of the dictionary of dataframes 1) as it is now with the zeros, 2) with the zeros dropped. 

If I keep the zeros I can retain more data and possibly have a better analysis for one of the cities that is not missing very much data. 

With the zeros dropped I can take a more accurate average of the cities, but I will be losing data points for certain dates. This might give me a better view of the trend in major cities overall, but will be missing data over certain periods of time for some of the produce.

In [13]:
drop_zeros_dict = {}

for name in names:
    index=0
    indices_to_drop = []
    df = adj2020_df_dict[name]
    for row in df.values:
        for element in row:
            if element == 0:
                indices_to_drop.append(adj2020_df_dict[name].index[index])
        index+=1
        
    drop_zeros_dict[name] = adj2020_df_dict[name].drop(indices_to_drop)

In [14]:
drop_zeros_dict['Potatoes']

,FarmPrice,AtlantaPrice,ChicagoPrice,LAPrice,NYCPrice
Date,,,,,
2019-05-05,1.61,5.11,5.18,4.02,6.27
2019-04-28,1.62,5.02,5.19,4.03,6.28
2019-04-21,1.62,5.02,4.35,4.03,6.28
2019-04-14,1.62,5.12,5.02,4.03,6.28
2019-04-07,1.62,4.11,5.02,4.03,6.28
...,...,...,...,...,...
2001-03-11,1.19,4.38,4.89,3.91,5.48
2001-03-04,0.95,4.74,4.63,3.72,4.38
2001-02-25,0.95,4.21,5.27,3.36,4.87


In [15]:
for name in names:
    print('Percentage of Invalid Data for', name)
    print('FarmPrice: ', round(sum(drop_zeros_dict[name]['FarmPrice'] <= 0)/len(drop_zeros_dict[name])*100, 2), '%')
    print('ATL: ', round(sum(drop_zeros_dict[name]['AtlantaPrice'] <= 0)/len(drop_zeros_dict[name])*100, 2), '%')
    print('CHI: ', round(sum(drop_zeros_dict[name]['ChicagoPrice'] <= 0)/len(drop_zeros_dict[name])*100, 2), '%')
    print('LA: ', round(sum(drop_zeros_dict[name]['LAPrice'] <= 0)/len(drop_zeros_dict[name])*100, 2), '%')
    print('NYC: ', round(sum(drop_zeros_dict[name]['NYCPrice'] <= 0)/len(drop_zeros_dict[name])*100, 2), '%')
    print('\n')

Percentage of Invalid Data for Broccoli Bunches
FarmPrice:  0.0 %
ATL:  0.0 %
CHI:  0.0 %
LA:  0.0 %
NYC:  0.0 %


Percentage of Invalid Data for Avocados
FarmPrice:  0.0 %
ATL:  0.0 %
CHI:  0.0 %
LA:  0.0 %
NYC:  0.0 %


Percentage of Invalid Data for Strawberries
FarmPrice:  0.0 %
ATL:  0.0 %
CHI:  0.0 %
LA:  0.0 %
NYC:  0.0 %


Percentage of Invalid Data for Romaine Lettuce
FarmPrice:  0.0 %
ATL:  0.0 %
CHI:  0.0 %
LA:  0.0 %
NYC:  0.0 %


Percentage of Invalid Data for Red Leaf Lettuce
FarmPrice:  0.0 %
ATL:  0.0 %
CHI:  0.0 %
LA:  0.0 %
NYC:  0.0 %


Percentage of Invalid Data for Potatoes
FarmPrice:  0.0 %
ATL:  0.0 %
CHI:  0.0 %
LA:  0.0 %
NYC:  0.0 %


Percentage of Invalid Data for Oranges
FarmPrice:  0.0 %
ATL:  0.0 %
CHI:  0.0 %
LA:  0.0 %
NYC:  0.0 %


Percentage of Invalid Data for Iceberg Lettuce
FarmPrice:  0.0 %
ATL:  0.0 %
CHI:  0.0 %
LA:  0.0 %
NYC:  0.0 %


Percentage of Invalid Data for Green Leaf Lettuce
FarmPrice:  0.0 %
ATL:  0.0 %
CHI:  0.0 %
LA:  0.0 %
NYC:  0.

Now to make an averages column

In [16]:
drop_zeros_dict['Strawberries']

,FarmPrice,AtlantaPrice,ChicagoPrice,LAPrice,NYCPrice
Date,,,,,
2019-05-05,0.58,2.44,1.90,2.14,2.93
2019-04-28,0.59,2.82,1.97,2.40,2.99
2019-04-21,0.70,2.95,2.06,2.40,3.19
2019-04-14,0.91,3.07,2.33,2.52,3.44
2019-04-07,1.09,3.07,2.97,2.64,3.78
...,...,...,...,...,...
2001-03-11,2.75,4.03,3.78,5.84,4.01
2001-02-25,2.89,4.58,5.12,5.37,4.58
2001-02-18,2.93,4.02,5.12,4.02,4.95


In [17]:
for name in names:
    drop_zeros_dict[name]['AvgRetail'] = np.mean(drop_zeros_dict[name].iloc[:, [1, 2, 3, 4]], axis=1)
    drop_zeros_dict[name]['AvgRetailVar'] = np.var(drop_zeros_dict[name].iloc[:, [1, 2, 3, 4]], axis=1)



/Users/timchristy/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/timchristy/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [18]:
drop_zeros_dict['Potatoes']

,FarmPrice,AtlantaPrice,ChicagoPrice,LAPrice,NYCPrice,AvgRetail,AvgRetailVar
Date,,,,,,,
2019-05-05,1.61,5.11,5.18,4.02,6.27,5.1450,0.633425
2019-04-28,1.62,5.02,5.19,4.03,6.28,5.1300,0.637050
2019-04-21,1.62,5.02,4.35,4.03,6.28,4.9200,0.744150
2019-04-14,1.62,5.12,5.02,4.03,6.28,5.1125,0.635869
2019-04-07,1.62,4.11,5.02,4.03,6.28,4.8600,0.823350
...,...,...,...,...,...,...,...
2001-03-11,1.19,4.38,4.89,3.91,5.48,4.6650,0.341525
2001-03-04,0.95,4.74,4.63,3.72,4.38,4.3675,0.156769
2001-02-25,0.95,4.21,5.27,3.36,4.87,4.4275,0.523119


In [19]:
adj2020_df_dict['Potatoes']

,FarmPrice,AtlantaPrice,ChicagoPrice,LAPrice,NYCPrice
Date,,,,,
2019-05-05,1.61,5.11,5.18,4.02,6.27
2019-04-28,1.62,5.02,5.19,4.03,6.28
2019-04-21,1.62,5.02,4.35,4.03,6.28
2019-04-14,1.62,5.12,5.02,4.03,6.28
2019-04-07,1.62,4.11,5.02,4.03,6.28
...,...,...,...,...,...
1999-08-23,1.24,6.01,0.00,3.46,5.00
1999-08-16,3.55,6.08,0.00,3.46,5.00
1999-08-09,1.81,6.16,0.00,3.58,5.70


In [20]:
savepath =  '../../data/02_processed/'

In [21]:
for name in names:
    adj2020_df_dict[name].to_pickle(savepath + name + '_2020_adj.pkl')
    drop_zeros_dict[name].to_pickle(savepath + 'no_zeros_' + name + '_2020_adj.pkl')
    